In [ ]:
""" This notebook will be the formal to train, analyze the word embedding data
    (with some ugly code temperately existed of course - but will be cleaned eventually!)
"""
import os
import re
import json
import itertools
from collections import defaultdict
from pprint import pprint
from datetime import datetime

import pandas as pd
import numpy as np

from gensim.models import KeyedVectors

from tc_corpus import TopCoderCorpus
from tc_pricing_models import train_word2vec_model, reduce_wv_dimensions, plot_word2vec, cosine_similarity, doc_vector_from_word_vectors

pd.set_option('display.max_rows', 300)
pd.set_option('display.width', 1000)

TRAINED_WV_PATH = os.path.join(os.curdir, 'models', 'model_20200510T145859')

In [ ]:
topcoder_corpus = TopCoderCorpus('detail_requirements.json')

In [ ]:
# Un-comment following line if training a new model
# trained_wv = train_word2vec_model(sentences=topcoder_corpus.get_challenge_req_sentences())

# Un-comment following line if using a trained model
trained_wv = KeyedVectors.load(TRAINED_WV_PATH)

the number of 2-element combinations of `cha_vec_dct` with replacement is 25336521, each execution of `cosine_similarity` takes 15.3 µs ± 1.74 µs (`timeit` with 100 runs and 10000 loops per run). To calculate all cosine similarity for every pair of documents that's stored in a default dict

```python
cha_cos_sim_matrix = defaultdict(dict)
for cha_a, cha_b in itertools.combinations_with_replacement(cha_vec_dct.keys(), 2):
    cha_cos_sim_matrix[cha_a][cha_b] = cha_cos_sim_matrix[cha_b][cha_a] = cosine_similarity(cha_vec_dct[cha_a], cha_vec_dct[cha_b])
```

it takes sometime between 380 and 390 seconds
      
Whereas the DOK format dict for cosine similarity

```python
cha_cos_sim_dok = {
    (cha_a, cha_b): cosine_similarity(cha_vec_dct[cha_a], cha_vec_dct[cha_b])
    for cha_a, cha_b in itertools.combinations_with_replacement(cha_vec_dct.keys(), 2)
}
```

takes 360 seconds.

## **Every second counts** ;-)

In [ ]:
# calculate the vector representation of each challenge, store it in a dictionary
challenge_req = topcoder_corpus.get_challenge_req_sentences(as_dataframe=True)
cha_vec_dct = {cha_id: doc_vector_from_word_vectors(cha['requirements'], trained_wv) for cha_id, cha in challenge_req.to_dict(orient='index').items()}

# calculate cosine similarity for every pair of challenges, sotre it in a DOK format dictionary
cha_cos_sim_dok = {
    (cha_a, cha_b): cosine_similarity(cha_vec_dct[cha_a], cha_vec_dct[cha_b])
    for cha_a, cha_b in itertools.combinations_with_replacement(cha_vec_dct.keys(), 2)
}


## Pricing model 0

Base on text mining and analogy estimation approach

- ✅ calculate Document2Vector of challenges
- ✅ calculate similarity between each pair of challenges
- for each task
  1. select 10 most similar tasks
  2. pricing strategies:
      - use average prize of 10 tasks as estimate prize of given
      - use mid prize of ...
  3. calculate estimation error (MRE magnitude of relative error) based on actual prize
  4. repeat
- on the entrie dataset, calculate mean MRE -> See how big is it, **this is the measure of accuraccy of the model**

## Pricing model 1

Once we get the MRE of all task, we can use any machine learning tech to analyze the error. e.g logistic regression

each ML approach produce one model

## Pricing model 2

Take the meta data of challenges/projects into consideration

- Types of challenges (aggregation & sub-model)

## BERT

> Nice to have application  
> Not only give a estimated price, but also reasons realted to recommended prize considerin the dynamic context.

Take a specific vector space, encode the hidden context, find the hidden factors of uncertainty of a given challenge.

In [ ]:
label_lst = ['java', 'python', 'javascript', 'php', 'mysql', 'api', 'design', 'prototype', 'ui', 'data', 'science']
wv_2D = reduce_wv_dimensions(trained_wv)

plot_word2vec(wv_2D, label_lst)